In [25]:
import numpy as np
import seaborn as sb

In [26]:
class Empleado():
    """
    Modela una actividad dentro del modelo de simulacion
    con una descripcion, una duracion minima, una duracion maxima
    y una duracion real establecida por una distribucion uniforme.
    """
    def __init__(self, tipo):
        if tipo == 1:
            self.nombre = "Empleado tipo 1"
            self.tiempoAtencion = np.random.normal(18,4) #Normal de media 18 minutos y Desv.est. de 4 minutos.
        elif tipo == 2:
            self.nombre = "Empleado tipo 2"
            self.tiempoAtencion = np.random.exponential(15) #Exponencial con tiempo promedio de 15 minutos. 
        elif tipo == 3:
            self.nombre = "Empleado tipo 3"
            self.tiempoAtencion = np.random.exponential(16) #Exponencial con promedio de 16 minutos.
        else:
            self.nombre = "Empleado tipo 4"
            self.tiempoAtencion = np.random.normal(14,3) #Normal de media 14 minutos y desv.est. de 3 minutos.
            


In [27]:
class Camion():
    def __init__(self, id_camion):
        self.id = id_camion
        self.tiempoLlegada = np.random.exponential(15)

In [28]:
class Surtidor():
    def __init__(self, tipo):
        self.ocupado = False
        self.tiempoOcupacion = 0
        self.tiempoAtencion = Empleado(tipo).tiempoAtencion
        
    def marcarLibre(self):
        self.ocupado = False
    
    def marcarOcupado(self):
        self.ocupado = True
    
    def actualizarPorcentajeOcupacion(self, reloj):
        self.tiempoOcupacion = self.tiempoOcupacion + reloj

In [29]:
class Evento():
    """
    nombre = puede ser ARRIBO_CAMION, FIN_ATENCION
    objeto = puede ser un Camion o un Surtidor
    duracion = duracion del evento
    """
    def __init__(self, nombre,objeto, duracion):
        self.nombre = nombre
        self.duracion = duracion
        self.objeto = objeto     

In [30]:
def inicializarModelo():
    fel = []
    lista_surtidores = []
    cantidad = 24 #24 horas
    
    for id_camion in range(cantidad): 
        camion = Camion(id_camion)
        fel.append(Evento("ARRIBO_CAMION",camion, camion.tiempoLlegada))
    
    for i in range(1,5):
        lista_surtidores.append(Surtidor(i))
        
    return fel, lista_surtidores    

In [102]:
def haySurtidorLibre(lista_surtidores):
    return not any(surtidor.ocupado  for surtidor in lista_surtidores)

In [32]:
def getSurtidorLibre(lista_surtidores):
    for surtidor in lista_surtidores:
        if (not surtidor.ocupado):
            return surtidor

In [76]:
def removerEvento(fel,evento): #remueve el siguiente evento de la FEL
    try:
        fel.pop()
    except:
        pass

In [82]:
def agregarEvento(fel,evento): #agrega un nuevo evento y ordena la FEL
    fel.append(evento)
    fel.sort(key=lambda evento: evento.duracion)

In [119]:
cant_experimentos = 60
cant_corridas = 100
reloj_simulacion = 0 
fel,lista_surtidores = inicializarModelo()
lista_promedio_atencion = []

for i in range(cant_experimentos):
    for j in range(cant_corridas):
        for evento in fel:
            reloj_simulacion = reloj_simulacion + evento.duracion
            if (evento.nombre == "ARRIBO_CAMION"):
                if haySurtidorLibre(lista_surtidores):
                    surtidor = getSurtidorLibre(lista_surtidores)
                    surtidor.marcarOcupado()
                    evento_inicio_atencion = Evento("INICIO_ATENCION",surtidor, surtidor.tiempoAtencion)
                    agregarEvento(fel, evento_inicio_atencion)
                    surtidor.actualizarPorcentajeOcupacion(tiempo_simulacion)
                    tiempoAtencionCamion = evento_inicio_atencion.duracion - evento.duracion
                    lista_promedio_atencion.append(tiempoAtencionCamion)
                else:
                    #TODO: no se que hacer aca para "simular" que un camion debe esperar en la cola
                    #cuando no hay surtidores libres, saco el evento "ARRIBO_CAMION" de la FEL?
                    pass
            if (evento.nombre == "INICIO_ATENCION"):
                evento_fin_atencion = Evento("FIN_ATENCION") #TODO: que duracion tiene este evento?
                agregarEvento(fel, evento_fin_atencion)
                
            elif (evento.nombre == "FIN_ATENCION"):
                evento.objeto.marcarLibre() #el objeto del evento es un surtidor
                evento.objeto.actualizarPorcentajeOcupacion(0) #Si el surtidor esta ocioso entonces no sumo nada a su tiempo de ocupacion


TypeError: __init__() missing 2 required positional arguments: 'objeto' and 'duracion'